# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [63]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [64]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [65]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [66]:
L, u = diy_lu(a)

print(a)
print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [67]:
a1 = a.copy()
a1[1, 1] = 3
print(a)
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Resulting matix still has full rank, but the naive LU routine breaks down.

In [68]:
np.linalg.matrix_rank(a1)

6

In [69]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


<ipython-input-63-8918e3eddeb9>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-63-8918e3eddeb9>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-63-8918e3eddeb9>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-63-8918e3eddeb9>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [70]:
# ... ENTER YOUR CODE HERE ...
def check_lm(a):
    ''' Determine if the matrix has non-zero leading minors
    
        Creates a submatrix of leading minors and computes determinant. If
        one of the determinant values is zero, breaks and tells you. '''
    nrows = a.shape[0]
    ncols = a.shape[1]
    nrows += 1
    for i in range(nrows):
        if i == 0:
            # don't compute a determinant of a 1x1 matrix
            # check if the value of a11 is 0
            if a[i, i] == 0:
                print("Invalid Matrix leading minors!")
                return True
                break
        else:
            # create a sub matrix for computation out of leading minors
            sub_matrix = a[:i+1, :i+1]
            det = np.linalg.det(sub_matrix)
            if det == 0:
                print("Invalid Matrix leading minors! Implement pivoting!")
                return True
                break
    print("Matrix is valid and ready for LU decomposition!")
    return False
        
check_lm(a)
check_lm(a1)

Matrix is valid and ready for LU decomposition!
Invalid Matrix leading minors! Implement pivoting!


True

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [74]:
# ... ENTER YOUR CODE HERE ...
def swap_rows(a, pos1, pos2):
    t = a[pos1].copy()
    a[pos1] = a[pos2]
    a[pos2] = t
    
def subtract_rows(row, top_row):
    return row - (row[0] / top_row[0]) * top_row

def pivot(matrix):
    pos = np.argmax(list(map(abs, matrix.T[0])))
    if pos != 0:
        swap_rows(matrix, 0, pos)

    matrix[1:,] = np.apply_along_axis(subtract_rows, 1, matrix[1:], matrix[0])

    return matrix
        
def diy_lu_pivot(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    Adds column pivoting if required.
    """
    NeedPivot = check_lm(a)
    if NeedPivot:
        print("Implementing pivoting...")
        for i in range(a.shape[0] - 1):
            a[i:, i:] = pivot(a[i:, i:])
        print("Pivoting complete")
        
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
        
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

print(a1)
diy_lu_pivot(a1)

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  2.220e-16  0.000e+00 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00  0.000e+00  2.776e-17  0.000e+00  0.000e+00 -1.233e-05]]
Matrix is valid and ready for LU decomposition!


(array([[ 1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00],
        [ 0.000e+00,  1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00],
        [ 0.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00],
        [ 0.000e+00, -9.869e-17,  1.551e-16,  1.000e+00,  0.000e+00,
          0.000e+00],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,
          0.000e+00],
        [ 0.000e+00,  0.000e+00, -1.696e-17,  3.537e-16,  9.015e-15,
          1.000e+00]]),
 array([[ 3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,
          3.000e+00],
        [ 0.000e+00, -2.250e+00, -2.571e+00, -2.700e+00, -2.769e+00,
         -2.812e+00],
        [ 0.000e+00,  0.000e+00, -1.636e+00, -1.929e+00, -2.118e+00,
         -2.250e+00],
        [ 0.000e+00,  1.233e-32,  0.000e+00, -9.247e-02, -1.485e-01,
         -1.856e-01],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.841e-03,
          3.821e-03

In [78]:
from scipy.linalg import lu

def reconstruct(matrix):
    
    """Reconstruct the Orignial given matrix using scipy.linalg.lu
    
        Note: Since it was not a requirement to use the orginal diy_lu function,
        this method was implemented using library fucntions and recomputes the 
        LU matrix prior to reconstruction.
        
        It should also be noted that this function assumes you have provided a 
        matrix (computed above) and does not check the leading minors.""" 
    print("Original Matrix:")
    print(matrix)
    P, L, U = lu(matrix)
    print("Permutation Matrix:")
    print(P)
    print("Lower Triangular Matrix:")
    print(L)
    print("Upper Triangular Matrix:")
    print(U)
    new_matrix = P.dot(L).dot(U)
    print("Reconstructed Matrix:")
    return new_matrix
    
reconstruct(a1)

Original Matrix:
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  2.220e-16  0.000e+00 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00  0.000e+00  2.776e-17  0.000e+00  0.000e+00 -1.233e-05]]
Permutation Matrix:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
Lower Triangular Matrix:
[[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -9.869e-17  1.551e-16  1.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00  1.000e+00  0.000e+00]
 [ 0.000e+00 -0.000e+00 -1.696e-17  3.537e-16  9.01

array([[ 3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,
         3.000e+00],
       [ 0.000e+00, -2.250e+00, -2.571e+00, -2.700e+00, -2.769e+00,
        -2.812e+00],
       [ 0.000e+00,  0.000e+00, -1.636e+00, -1.929e+00, -2.118e+00,
        -2.250e+00],
       [ 0.000e+00,  2.220e-16,  2.469e-32, -9.247e-02, -1.485e-01,
        -1.856e-01],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.841e-03,
         3.821e-03],
       [ 0.000e+00,  0.000e+00,  2.776e-17, -1.689e-33,  1.277e-33,
        -1.233e-05]])